#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Contando palabras: Construye una aplicacion que cuente palabras de forma eficiente

Este laboratorio usara las tecnologias descritas en los materiales del curso sobre Spark para desarrollar una aplicacion de conteo de palabras. 

Con el uso masivo de Internet y las redes sociales, el volumen de texto no estructurado esta creciendo dramaticamente, y Spark es una gran herramienta para analizar este tipo de datos. En esta PEC, vamos a escribir codigo para encontrar las palabras mas comunes en las [obras completas de William Shakespeare](http://www.gutenberg.org/ebooks/100) recuperados a partir de [Proyecto Gutenberg](http://www.gutenberg.org/wiki/Main_Page).

Lo mas interesante de la forma de trabajar en esta practica es que podria escalarse para, por ejemplo, encontrar las palabras mas comunes en Wikipedia.

## Durante esta PEC vamos a cubrir:

* *Parte 1:* Creacion de un RDD y un pair RDD
* *Parte 2:* Contar palabras usando un pair RDD
* *Parte 3:* Encontrar las palabras individuales y su frecuencia de aparicion media
* *Parte 4:* Aplicar las funcionalidades desarrolladas a un archivo de texto* 
* *Parte 5:* Calcular algunos estadisticos*


> Como referencia a todos los detalles de los metodos que se usan en esta practica usar:
> * [API Python de Spark](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

## Parte 1: Creacion de un RDD y un pair RDDs

En esta seccion, exploraremos como como crear RDDs usando `parallelize` y como aplicar pair RDDs al problema del conteo de palabras.

### (1a) Creacion de un RDD
Empezemos generando un RDD a partir de una lista de Python y el metodo `sc.parallelize`. Luego mostraremos por pantalla el tipo de la variable generada.

In [3]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
print type(wordsRDD)

<class 'pyspark.rdd.RDD'>

### (1b) Crear el plural de las palabas y testear

Vamos a utilizar una transformacion `map()` para incorporar la letra 's' a cada uno de los strings almacenados en el RDD que acabamos de crear. Vamos a definir una funcion de Python que devuelva una palabra, que se le ha pasado como parametro, incorporando una "s" al final de la misma. Reemplazar el texto `<FILL IN>` con la solucion propuesta. Despues de haber definido correctamente la funcion `makePlural`, ejecutar la segunda celda que contiene un assert de test. Si la solucion es correcta, se imprimira `1 test passed`.

Esta sera la forma habitual de trabajar en las PECs. Los ejercicios contendran una explicacion de lo que se espera, seguido de una celda de codigo con uno o mas `<FILL IN>`. Las celdas que necesiten ser modificadas contendran el texto `# TODO: Replace <FILL IN> with appropriate code` en la primera linea.

Una vez se hayan sustituido todos los `<FILL IN>` por el codigo Python adecuado, ejecutar la celda, y posteriormente ejecutar la celda siguiente de test para comprobar que que la solucion es la esperada.

In [5]:
# TODO: Replace <FILL IN> with appropriate code
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

print makePlural('cat')

cats

In [6]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from databricks_test_helper import Test
# TEST Pluralize and test (1b)
Test.assertEquals(makePlural('rat'), 'rats', 'incorrect result: makePlural does not add an s')

1 test passed.

### (1c) Aplicar `makePlural` a nuestro RDD

Ahora es el momento de aplicar nuestra funcion `makePlural()` a todos los elementos del RDD usando una transformacion [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). Posteriormente ejecutar la accion [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) para obtener el RDD transformado.

In [8]:
# TODO: Replace <FILL IN> with appropriate code
pluralRDD = wordsRDD.map(makePlural)
print pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

In [9]:
# TEST Apply makePlural to the base RDD(1c)
Test.assertEquals(pluralRDD.collect(), ['cats', 'elephants', 'rats', 'rats', 'cats'],
                  'incorrect values for pluralRDD')

1 test passed.

### (1d) Ejecutar una funcion `lambda` en un `map`

Vamos a crear el mismo RDD usando una `lambda` function en lugar de una funcion con nombre.

In [11]:
# TODO: Replace <FILL IN> with appropriate code
pluralLambdaRDD = wordsRDD.map(lambda word: word + 's')
print pluralLambdaRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']

In [12]:
# TEST Pass a lambda function to map (1d)
Test.assertEquals(pluralLambdaRDD.collect(), ['cats', 'elephants', 'rats', 'rats', 'cats'],
                  'incorrect values for pluralLambdaRDD (1d)')

1 test passed.

### (1e) Numero de caracteres de cada una de las palabras

Ahora vamos a usar un `map()` y una funcion lambda `lambda` para obtener el numero de caracteres de cada palabra. Usaremos `collect` para guardar este resultado directamente en una variable.

In [14]:
# TODO: Replace <FILL IN> with appropriate code
pluralLengths = (pluralRDD
                 .map(lambda word: len(word))
                 .collect())
print pluralLengths

[4, 9, 4, 4, 4]

In [15]:
# TEST Length of each word (1e)
Test.assertEquals(pluralLengths, [4, 9, 4, 4, 4],
                  'incorrect values for pluralLengths')

1 test passed.

### (1f) Pair RDDs

El siguiente paso para completar nuestro programa de conteo de palabras en crear un nuevo tipo de RDD, llamado pair RDD. Un pair RDD es un RDD donde cada elemento es un tupla del estilo `(k, v)` donde `k` es la clave y `v` es su valor correspondiente. En este ejemplo, crearemos una pair RDD consistente en tuplas con el formato `('<word>', 1)` para cada elemento de nuestro RDD basico.

Podemos crear nuestro pair RDD usando una transformacion `map()` con una `lambda()` function que cree un nuevo RDD.

In [17]:
# TODO: Replace <FILL IN> with appropriate code
wordPairs = wordsRDD.map(lambda word: (word, 1))
print wordPairs.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]

In [18]:
# TEST Pair RDDs (1f)
Test.assertEquals(wordPairs.collect(),
                  [('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)],
                  'incorrect value for wordPairs')

1 test passed.

## Parte 2: Contar palabras usando un pair RDD

Ahora, contaremos el numero de veces que una palabra en particular aparece en el RDD. Esta operacion se puede realizar de una infinidad de maneras, pero algunas seran mucho menos eficientes que otras.

Un solucion muy sencilla seria usar `collect()` sobre todos los elementos devolverlos al driver y alli contarlos. Mientras esta forma de trabajar podria funcionar con textos relativamente cortos, nosotros lo que queremos es poder trabajar con textos de cualquier longitud. Adicionalmente, ejecutar todo el calculo en el driver es mucho mas lento que ejecutarlo en paralelo en los workers. Por estos motivos, en esta practica usaremos operaciones paralelizables.

%md
### (2a) Usando `groupByKey()`
Una primera solucion a nuestro problema, luego veremos que hay otras mucho mas eficientes, se podria basar en la transformacion [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformacion `groupByKey()` agrupa todos los elementos de un RDD que compartan la misma clave en una unica lista dentro de una de las particiones.

Esta operacion plantea dos problemas:
  + Esta operacion necesita mover todos los valores dentro de la particion adecuada. Esto satura la red. 
  + Las listas generadas pueden llegar a ser muy grandes llegando incluso a saturar la memoria de alguno de los trabajadadores
  
Utiliza `groupByKey()` para generar un pair RDD del tipo `('word', iterator)`.

In [20]:
# TODO: Replace <FILL IN> with appropriate code
# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0}: {1}'.format(key, list(value))

rat: [1, 1]
elephant: [1]
cat: [1, 1]

In [21]:
# TEST groupByKey() approach (2a)
Test.assertEquals(sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()),
                  [('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])],
                  'incorrect value for wordsGrouped')

1 test passed.

### (2b) Utiliza `groupByKey()` para obtener los conteos

Usando la transformacion `groupByKey()` crea un RDD que contenga 2 elementos, donde cada uno de ellos sea un par palabra (clave) iterador de Python (valor).

Luego suma todos los valores de iterador usando una transformacion `map()`. El resultado debe ser un pair RDD que contenga las parejas (word, count).

In [23]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsGrouped = wordsGrouped.mapValues(len)
print wordCountsGrouped.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]

In [24]:
# TEST Use groupByKey() to obtain the counts (2b)
Test.assertEquals(sorted(wordCountsGrouped.collect()),
                  [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCountsGrouped')


1 test passed.

** (2c) Conteo usando `reduceByKey` **

Una mejor solucion es comenzar desde un pair RDD y luego usar la transformacion [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) para crear un nuevo pair RDD. La transformacion `reduceByKey()` agrupa todas las parejas que comparten la misma clave. Posteriormente aplica la funcion que se le pasa por parametro agrupando los valores de dos en dos. Este proceso se repite iterativamente hasta que obtenemos un unico valor agregado para cada una de las claves del pair RDD. `reduceByKey()` opera aplicando la funcion primero dentro de cada una de las particiones de forma independiente, y posteriormente unicamente comparte los valores agregados entre particiones diferentes, permitiendole escalar de forma eficiente ya que no tiene necesidad de desplazar por la red una gran cantidad de datos.

In [26]:
# TODO: Replace <FILL IN> with appropriate code
# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print wordCounts.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]

In [27]:
# TEST Counting using reduceByKey (2c)
Test.assertEquals(sorted(wordCounts.collect()), [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCounts')

1 test passed.

### (2d) Ahora todo junto

La version mas compleja del codigo ejecuta primero un `map()` sobre el pair RDD, la transformacion `reduceByKey()`, y finalmente la accion `collect()` en una unica linea de codigo.

In [29]:
# TODO: Replace <FILL IN> with appropriate code
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda x, y: x + y)
                       .collect())
print wordCountsCollected

[('rat', 2), ('elephant', 1), ('cat', 2)]

In [30]:
# TEST All together (2d)
Test.assertEquals(sorted(wordCountsCollected), [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect value for wordCountsCollected')

1 test passed.

## Parte 3: Encontrar las palabras individuales y su frecuencia de aparicion media

### (3a) Palabras unicas

Calcular el numero de palabras unicas en `wordsRDD`. Puedes utitlziar otros RDDs que hayas creado en esta practica si te resulta mas sencillo.

In [32]:
# TODO: Replace <FILL IN> with appropriate code
uniqueWords = len(wordsRDD.distinct().collect())
#Another feasible approach: uniqueWords = len(set(wordsRDD.collect()))
print uniqueWords

3

In [33]:
# TEST Unique words (3a)
Test.assertEquals(uniqueWords, 3, 'incorrect count of uniqueWords')

1 test passed.

### (3b) Calular la media usando `reduce()`

Encuentra la frequencia media de aparicion de palabras en `wordCounts`.

Utiliza la accion `reduce()` para sumar los conteos en `wordCounts` y entonces divide por el numero de palabras unicas. Para realizar esto primero aplica un `map()` al pair RDD `wordCounts`, que esta formado por tuplas con el formato (key, value), para convertirlo en un RDD de valores.

In [35]:
# TODO: Replace <FILL IN> with appropriate code
from operator import add
totalCount = (wordCounts
              .map(lambda keyValue: keyValue[1])
              .reduce(add))
average = totalCount / float(uniqueWords)
print totalCount
print round(average, 2)

5
1.67

In [36]:
# TEST Mean using reduce (3b)
Test.assertEquals(round(average, 2), 1.67, 'incorrect value of average')

1 test passed.

## Parte 4: Aplicar las funcionalidades desarrolladas a un archivo de texto

Para esto hemos de construir una funcion `wordCount`, capaz de trabajar con datos del mundo real que suelen presentan problemas como el uso de mayusculas o minusculas, puntuacion, acentos, etc. Posteriormente, cargar los datos de nuestra fuente de datos y finalmente, calular el conteo de palabras sobre los datos procesados.

### (4a) funcion `wordCount`

Primero, define una funcion para el conteo de palabras. Deberias reusar las tecnicas que has visto en los apartados anteriores de esta practica. Dicha funcion, ha de tomar un RDD que contenga una lista de palabras, y devolver un pair RDD que contenga todas las palabras con sus correspondientes conteos.

In [38]:
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return wordListRDD.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
                       
print wordCount(wordsRDD).collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]

In [39]:
# TEST wordCount function (4a)
Test.assertEquals(sorted(wordCount(wordsRDD).collect()),
                  [('cat', 2), ('elephant', 1), ('rat', 2)],
                  'incorrect definition for wordCount function')

[('rat', 2), ('elephant', 1), ('cat', 2)]
1 test passed.

### (4b) Mayusculas y puntuacion

Los ficheros del mundo real son mucho mas complejos que los que hemos estado usando en esta PAC. Algunos de los problemas que son necesarios de solucionar son:
  + Las palabras deben de contarse independientemente de si estan en mayuscula o minuscula (por ejemplo, Spark y spark deberian contarse como la misma palabra).
  + Todos los signos de puntuacion han de eliminarse.
  + Cualquier espacio al principio o al final de la palabra ha de eliminarse.
  
Define la funcion `removePunctuation` que convierta todo el texto a minusculas, elimine los signos de puntuacion, y elimine los espacios al principio y fin de cada palabra. Usa el modulo de Python [re](https://docs.python.org/2/library/re.html) para eliminar cualquier caracter que no sea una letra, un numero o un espacio.

Sino estas familiarizado con las expresiones regulares deberias revisar [este tutorial](https://developers.google.com/edu/python/regular-expressions). Alternativamente, [esta web](https://regex101.com/#python) es de gran ayuda para debugar tus expresiones regulares.

**Hints**

1. Usa la funcion [re.sub()](https://docs.python.org/2.7/library/re.html#re.sub).
2. Para nuestros propositos, "puntuacion" significa "no alphabetico, numerico, o espacio." La expresion regular que define estos caracteres es: `[^A-Za-z\s\d]`
3. No usar `\W`, ya que retendra los guiones bajos.

In [41]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub('[^A-Za-z0-9\s]+', '', text.lower()).strip()
    
print removePunctuation('Hi, you!')
print removePunctuation(' No under_score!')
print removePunctuation(' *      Remove punctuation then spaces  * ')

hi you
no underscore
remove punctuation then spaces

In [42]:
# TEST Capitalization and punctuation (4b)
Test.assertEquals(removePunctuation(" The Elephant's 4 cats. "),
                  'the elephants 4 cats',
                  'incorrect definition for removePunctuation function')

1 test passed.

### (4c) Cargar un fichero de texto

Para la siguiente parte, usaremos las [Obras completas de William Shakespeare](http://www.gutenberg.org/ebooks/100) del [Proyecto Gutenberg](http://www.gutenberg.org/wiki/Main_Page). Para convertir un fichero de texto en un RDD, usaremos el metodo `SparkContext.textFile()`. Tambien usaremos la funcion que acabamos de crear `removePunctuation()` dentro de una transformacion `map()` para eliminar todos los caracteres no alphabeticos, numericos or espacios. Dado que el fichero es bastante grandre, usaremos `take(15)`, de forma que tan solo imprimiremos por pantalla las 15 primeras lineas.

In [44]:
%fs

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

In [45]:
# Tan solo ejecuta este codigo
import os.path
fileName = "dbfs:/" + os.path.join('databricks-datasets', 'cs100', 'lab1', 'data-001', 'shakespeare.txt')

shakespeareRDD = sc.textFile(fileName, 8).map(removePunctuation)
print '\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda (l, num): '{0}: {1}'.format(num, l))  # to 'lineNum: line'
                .take(15))

0: 1609
1: 
2: the sonnets
3: 
4: by william shakespeare
5: 
6: 
7: 
8: 1
9: from fairest creatures we desire increase
10: that thereby beautys rose might never die
11: but as the riper should by time decease
12: his tender heir might bear his memory
13: but thou contracted to thine own bright eyes
14: feedst thy lights flame with selfsubstantial fuel

### (4d) Extraer las palabras de las lineas

Antes de poder usar la funcion `wordcount()`, hemos de solucionar dos problemas con el formato del RDD:
  + El primer problema es que necesitamos dividir cada linea por sus espacios. ** Esto lo solucionaremos en el apartado (4d). **
  + El segundo problema es que necesitamos filtar las lineas completamente vacias. ** Esto lo solucionaremos en el apartado (4e). **

Para aplicar una transformacion que divida cada elemento del RDD por sus espacios, hemos de aplicar la funcion incorporada en los strings de Python [split()](https://docs.python.org/2/library/string.html#string.split). Cuidado que a primera vista puede parecer que la funcion necesaria es una transformacion `map()`, pero si piensas un poco mas sobre el resultado de la funcion `split()` te daras cuenta que esta no es la opcion correcta.

> Nota:
> * No uses la implementacion estandar del `split()`, pasale un valor de separacion. Por ejemplo, para dividir `line` por comas, usa `line.split(',')`.

In [47]:
# TODO: Replace <FILL IN> with appropriate code
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda line: line.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()
print shakespeareWordsRDD.top(5)
print shakespeareWordCount

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']
927631

In [48]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after
# punctuation is removed.
Test.assertTrue(shakespeareWordCount == 927631 or shakespeareWordCount == 928908,
                'incorrect value for shakespeareWordCount')
Test.assertEquals(shakespeareWordsRDD.top(5),
                  [u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds'],
                  'incorrect value for shakespeareWordsRDD')

1 test passed.
1 test passed.

### (4e) Elimina los elementos vacios

El siguiente paso es eliminar los espacios vacios. Elimina todas las entradas donde la palabra sea `''`.

In [50]:
# TODO: Replace <FILL IN> with appropriate code
shakeWordsRDD = shakespeareWordsRDD.filter(bool) # Fuente: https://stackoverflow.com/a/41395487/5946191
shakeWordCount = shakeWordsRDD.count()
print shakeWordCount

882996

In [51]:
# TEST Remove empty elements (4e)
Test.assertEquals(shakeWordCount, 882996, 'incorrect value for shakeWordCount')

1 test passed.

### (4f) Cuenta las palabras

Ahora que tenemos un RDD que contiene solo palabras. El siguiente paso es aplicar la funcion `wordCount()` para producir una lista con los conteos de palabras. Podemos ver las 15 mas comunes usando la accion `takeOrdered()`; sin embargo, como los elementos del RRD son pares, necesitamos una funcion especial que ordene los pares de la forma correcta.

Usa las funciones  `wordCount()` y `takeOrdered()` para obtener las 15 palabras mas comunes junto con sus conteos.

In [53]:
# TODO: Replace <FILL IN> with appropriate code
top15WordsAndCounts = wordCount(shakeWordsRDD).takeOrdered(15, key = lambda x: -x[1])
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts))

the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678

In [54]:
# TEST Count the words (4f)
Test.assertEquals(top15WordsAndCounts,
                  [(u'the', 27361), (u'and', 26028), (u'i', 20681), (u'to', 19150), (u'of', 17463),
                   (u'a', 14593), (u'you', 13615), (u'my', 12481), (u'in', 10956), (u'that', 10890),
                   (u'is', 9134), (u'not', 8497), (u'with', 7771), (u'me', 7769), (u'it', 7678)],
                  'incorrect value for top15WordsAndCounts')

1 test passed.

## Parte 5: Calcular algunos estadisticos

Ahora que en shakeWordsRDD tenemos un RDD que contiene solo palabras, vamos a intentar estudiar algunas caracteristicas de la lengua inglesa. Usando las mismas tecnicas que has aplicado en los ejercicios anteriores responde a las siguientes preguntas:

- ¿Cuantas palabras distintas tienen una longitud mayor de 5 letras y empiezan y acaban con la misma letra?
- ¿Cuantas palabras distintas tienen más de 4 vocales?
- ¿Cual es la palabra de seis letas que más se repite?¿Cuantas veces aparece?

In [56]:
# ¿Cuantas palabras distintas tienen una longitud mayor de 5 letras y empiezan y acaban con la misma letra?
moreThan5WithSameLetter = shakeWordsRDD.filter(lambda x: len(x) > 5).filter(lambda x: x[0]==x[len(x)-1])
distinctWords = wordCount(moreThan5WithSameLetter).distinct().collect()
print str(len(distinctWords)) + ' palabras distintas tienen una longitud mayor de 5 letras y empiezan y acaban con la misma letra'

1380 palabras distintas tienen una longitud mayor de 5 letras y empiezan y acaban con la misma letra

In [57]:
from collections import Counter
####################################
# ¿Cuantas palabras distintas tienen más de 4 vocales?
def countVowels(word):
  c = Counter(word)
  return c['a'] + c['e'] + c['i'] + c['o'] + c['u']
  
moreThan4Vowels = wordCount(shakeWordsRDD.filter(lambda word: len(word) >= 4).filter(lambda word: countVowels(word) > 4)).distinct().collect()
print str(len(moreThan4Vowels)) + ' palabras distintas tienen más de 4 vocales'

1801 palabras distintas tienen más de 4 vocales

In [58]:
####################################
# ¿Cual es la palabra de seis letas que más se repite?¿Cuantas veces aparece?
sixLetters = shakeWordsRDD.filter(lambda word: len(word) == 6)
wordSixLettersMostUsed = wordCount(sixLetters).takeOrdered(1, key = lambda x: -x[1])
print '\n'.join(map(lambda (w, c): '{0} es palabra de seis letras que más se repite, aparece {1} veces'.format(w, c), wordSixLettersMostUsed))

should es palabra de seis letras que más se repite, aparece 1572 veces